In [1]:
import requests

import statistics
import pandas as pd
import requests
import json
import numpy as np
import itertools

api_key ='8481ed700f2bf3bb0575f4e9d88f8bbf'
import datetime
import statistics

# Function to fetch P/E ratios for a specific date
def fetch_pe_ratios(api_key, date, exchange='NYSE'):
    url = f"https://financialmodelingprep.com/api/v4/industry_price_earning_ratio?date={date}&exchange={exchange}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Function to calculate the average P/E ratio over the last three years
def calculate_three_year_avg_pe(api_key, exchange='NYSE'):
    industry_pe_dict = {}
    
    # Get today's date
    today = datetime.date.today()
    
    # Iterate through the last three years
    for year_offset in range(5):
        date = today.replace(year=today.year - year_offset).strftime('%Y-%m-%d')
        data = fetch_pe_ratios(api_key, date, exchange)
        if data:
            for industry in data:
                industry_name = industry['industry']
                pe_ratio = float(industry['pe'])
                
                if industry_name not in industry_pe_dict:
                    industry_pe_dict[industry_name] = []
                industry_pe_dict[industry_name].append(pe_ratio)
    
    # Calculate the average P/E ratio for each industry
    avg_pe_ratios = {industry: statistics.median(pe_list) for industry, pe_list in industry_pe_dict.items()}

    
    return avg_pe_ratios

# Replace with your API key
api_key = '8481ed700f2bf3bb0575f4e9d88f8bbf'

# Calculate the average P/E ratio
average_pe_ratios = calculate_three_year_avg_pe(api_key)

# List of tickers to process
tickers =['AAPL', 'MSFT', 'GOOGL','MIDD','NN','STEM']  #['SMG']

# Dictionary to store results
results = []

# Function to scale scores using the tanh function
def tanh_scale(x):
  return (np.tanh(x) + 1) * 5  # Scale to 0 to 10

def power_scale(x, exponent=0.5):
    return np.power(x, exponent) * 10  # Exponent < 1 spreads small values, > 1 spreads large values

def scale_value(value, min_value, max_value):
    # Scale the value between 1 and 10 using min-max normalization
    scaled_value = 1 + 9 * (value - min_value) / (max_value - min_value)
    return scaled_value

import numpy as np

def log_scale(agg_score, max_value=60):
    # Apply logarithmic scaling and normalize to 0-10
    return (np.log(agg_score + 1) / np.log(max_value + 1)) * 10

def min_max_scale(value, min_value, max_value):
    return round(1 + 9 * (value - min_value) / (max_value - min_value), 1)

# Example usage for the aggregate health score:
min_value = 0  # Min possible value for the aggregated score
max_value = 60  # Max possible value (sum of 6 scores each ranging 0-10)



for tck in tickers:
  try:
      # Fetch financial ratios
      response1 = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{tck}?period=quarter&apikey={api_key}')
      response1.raise_for_status()
      df1 = pd.DataFrame(response1.json())
      
      # Calculate PE ratio
      pe_ratio = df1['priceEarningsRatio'][0]
      
      # Fetch income statement growth
      response2 = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement-growth/{tck}?period=annual&apikey={api_key}')
      response2.raise_for_status()
      df2 = pd.DataFrame(response2.json())
      
      # Calculate revenue growth
      revenue_growth = (df2['growthRevenue'][0]) * 100 #[0:3].mean()
      
      # Fetch company profile
      response3 = requests.get(f'https://financialmodelingprep.com/api/v3/profile/{tck}?apikey={api_key}')
      response3.raise_for_status()
      df3 = pd.DataFrame(response3.json())
      
      # Get industry
      industry = df3['industry'][0]
      
      # Calculate PE score
      percentage_diff = ((pe_ratio - average_pe_ratios.get(industry, 1)) * 100) / average_pe_ratios.get(industry, 1)
      pe_score = ((-1 * percentage_diff) + 1.3 * revenue_growth)
      scaled_pe_score = round(tanh_scale(pe_score),1)
      if pe_ratio<0:
          scaled_pe_score=0
      
      # Calculate Debt to Equity score
      debt_to_equity = df1['debtEquityRatio'][0]
      Debt_Score = ((-0.1 * debt_to_equity) + 1.3 * (revenue_growth / 10))
      Debt_Score = Debt_Score * 2
      scaled_debt_score = round(tanh_scale(Debt_Score),1)
      if debt_to_equity<0:
          scaled_debt_score=0
      
      # Calculate Dividend Yield score
      dividend_yield = df1['dividendYield'][0]
      Dividend_Yield_Score = ((0.2 * dividend_yield) + 1.3 * (revenue_growth / 10))
      scaled_dividend_Yield_Score = round(tanh_scale(Dividend_Yield_Score),1)
      if dividend_yield==0:
          scaled_dividend_Yield_Score=0
      
      # Calculate ROCE score
      roce = df1['returnOnCapitalEmployed'][0]
      profit_growth = (df2['growthNetIncome'][0]) * 100 #[0:3].mean()
      ROCE_Score = ((roce / 10) + (profit_growth / 10))
      scaled_roce_score = round(tanh_scale(ROCE_Score),1)
      if roce<0:
          scaled_roce_score=0
      
      # Calculate ROE score
      roe = df1['returnOnEquity'][0]
      roe_score = roe / 10 
      scaled_roe_score = round(tanh_scale(roe_score),1)
      if roe<0:
          scaled_roe_score=0

      
      # Calculate PEG ratio score
      peg = df1['priceEarningsToGrowthRatio'][0]
      peg_score = peg / 10 if peg > 0 else 0
      scaled_peg_score = round(tanh_scale(peg_score),1)

      # Calculate aggregate health score
      aggregate_health_score = (
          scaled_pe_score +
          scaled_debt_score +
          scaled_dividend_Yield_Score +
          scaled_roce_score +
          scaled_roe_score +
          scaled_peg_score
      )
      aggregate_health_score=min_max_scale(aggregate_health_score,min_value,max_value)
      
      # Store results
      results.append({
          'Ticker': tck,
          'Scaled PE Score': scaled_pe_score,
          'Scaled Debt Score': scaled_debt_score,
          'Scaled Dividend Yield Score': scaled_dividend_Yield_Score,
          'Scaled ROCE Score': scaled_roce_score,
          'Scaled ROE Score': scaled_roe_score,
          'Scaled PEG Score': scaled_peg_score,
          'Aggregate Health Score': aggregate_health_score

      })
      
  except Exception as e:
      print(f"Error processing {tck}: {e}")

# Convert results to DataFrame
results_df = pd.DataFrame(results)

In [2]:
results_df

,Ticker,Scaled PE Score,Scaled Debt Score,Scaled Dividend Yield Score,Scaled ROCE Score,Scaled ROE Score,Scaled PEG Score,Aggregate Health Score
0,AAPL,0.0,1.1,3.3,3.7,5.2,5.0,3.7
1,MSFT,10.0,10.0,9.8,9.9,5.0,10.0,9.2
2,GOOGL,10.0,9.9,9.1,9.9,5.0,5.0,8.3
3,MIDD,10.0,4.4,0.0,1.6,5.0,5.2,4.9
4,NN,0.0,2.3,0.0,0.0,0.0,5.2,2.1
5,STEM,0.0,0.0,0.0,0.0,6.4,5.0,2.7
